# Postman Collection Converter

This notebook demonstrates how to convert Postman collections to Python API client code using the PostmanCollectionConverter library.

In [1]:
import os

from models import PostmanCollection
from converter import PostmanCollectionConverter, PostmanRequestConverter
from tester import test_exports

## Load a Postman Collection

First, let's load a Postman collection from a JSON file.

In [2]:
# Replace this with your Postman collection path
collection_path = "test.postman_collection.json"

# Define the output directory
output_dir = "./EXPORT/domo_api"

# Load the collection from a file
collection = PostmanCollection.from_file(collection_path)
collection.list_all_headers()

{'host': ['domo-community.domo.com'],
 'accept': ['*/*', 'application/json, text/plain, */*'],
 'sec-ch-ua': ['"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"'],
 'user-agent': ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'],
 'content-type': ['application/json',
  'application/json;charset=utf-8',
  'application/json;charset=UTF-8'],
 'x-csrf-token': ['530f4e52-edef-49f8-9bb6-151535e22469'],
 'sec-fetch-dest': ['empty'],
 'sec-fetch-mode': ['cors'],
 'sec-fetch-site': ['same-origin'],
 'sec-ch-ua-mobile': ['?0'],
 'x-requested-with': ['XMLHttpRequest'],
 'sec-ch-ua-platform': ['"Windows"'],
 'x-domo-requestcontext': ['{"clientToe":"C5MAH5D893-J9NMQ"}',
  '{"clientToe":"C5MAH5D893-CKQE4"}',
  '{"clientToe":"C5MAH5D893-EBRDT"}',
  '{"clientToe":"C5MAH5D893-5AFLD"}',
  '{"clientToe":"C5MAH5D893-M0ASA"}',
  '{"clientToe":"C5MAH5D893-55M2H"}',
  '{"clientToe":"C5MAH5D893-0NV22"}',
  '{"clientToe":"C5MAH5D

## Customize the Conversion

Let's define some customization options for specific functions.

In [3]:
# Define default headers to include in all requests
default_required_headers = ['accept', 'content-type']

# Customization options for specific endpoints
customizations = {
    # Function name -> customization options
    "get_cards": {
        "default_params": [ "parts", "includeFiltered"],
        "ignored_params" : ["urns", "includeFiltered"],
    },
    "post_domoweb": {
    }
}

## Generate Python API Client Files

Now let's generate the Python API client files from the Postman collection.

In [4]:

converter = PostmanCollectionConverter.from_postman_collection(
    postman_path=collection_path,
    export_folder=output_dir,
    customize=customizations,
    required_headers=default_required_headers  # Pass global required headers
)
    
 

Exported function to ./EXPORT/domo_api\post_domoweb.py
Exported function to ./EXPORT/domo_api\get_cards.py
Exported function to ./EXPORT/domo_api\get_settings.py
Exported function to ./EXPORT/domo_api\get_general.py
Exported function to ./EXPORT/domo_api\get_domoopenai.py
Exported function to ./EXPORT/domo_api\get_shares.py
Exported function to ./EXPORT/domo_api\get_index.py
Exported function to ./EXPORT/domo_api\get_1728973208.py
Exported function to ./EXPORT/domo_api\post_query.py
Exported function to ./EXPORT/domo_api\post_search.py
Exported function to ./EXPORT/domo_api\get_9a2f8f7a_b0bd_49d6_8bc7_641453b3df24.py
Exported function to ./EXPORT/domo_api\get_permissions.py
Exported function to ./EXPORT/domo_api\get_101.py
Exported function to ./EXPORT/domo_api\get_providers.py
Exported function to ./EXPORT/domo_api\post_clog.py
Exported function to ./EXPORT/domo_api\post_query.py
Exported function to ./EXPORT/domo_api\get_818920ec_b891_4005_953d_3861403db86b.py
Exported function to ./

## Test the Generated API Client

Now we can test our generated API client functions.

In [ ]:
# Define auth credentials
auth = {
    "base_url": "https://domo-community.domo.com",
    "headers": {
        "x-domo-developer-token": os.environ["DOMO_ACCESS_TOKEN"],
        "base_url": f"https://{os.environ['domo_instance']}.domo.com",
    }
}



# Test the exported functions (commented out to avoid making actual API calls)
results = test_exports(
    export_folder=output_dir,
    auth=auth,
    debug_api=True,
    update_files=True
)

Testing module custom_post_domoweb from ./EXPORT/domo_api\custom_post_domoweb.py...
Error loading module custom_post_domoweb.py: positional argument follows keyword argument (custom_post_domoweb.py, line 25)
Testing module get_101 from ./EXPORT/domo_api\get_101.py...
Testing get_101.get_101...
🚀 Making get request to https://domo-community.domo.com/api/codeengine/v2/packages/9a2f8f7a-b0bd-49d6-8bc7-641453b3df24/versions/1.0.1
Headers: {'accept': 'application/json, text/plain, */*', 'content-type': 'application/json;charset=utf-8', 'x-domo-developer-token': '83ece44f1451d4b581e1191f98cd411164f0b5b6ad2755b3', 'base_url': 'https://domo-community.domo.com'}
Params: {'parts': 'functions,code'}
Data: None
JSON: None
  Status: 200
Running test function get_101.test_get_101...
  Status: 200
Running test function get_101.test_get_101...
  Result: 200
Updated get_101.py with test results
Testing module get_1728973208 from ./EXPORT/domo_api\get_1728973208.py...
Testing get_1728973208.get_17289732

## Individual Request Conversion

You can also convert individual Postman requests directly.

In [ ]:
if 'collection' in locals() and len(collection.requests) > 0:
    # Get a specific request from the collection
    test_request = collection.requests[0]
    
    print(f"Converting request: {test_request.name}")
    
    # Convert with specific configuration
    required_headers = ["content-type", "host"]
    default_params = ["urns", "parts"]  # Parameters to expose as function arguments
    ignored_params = ["cacheBuster"]  # Parameters to exclude
    
    # Create a converter for this request
    req_converter = PostmanRequestConverter(Request=test_request)
    
    # Process headers with filtering - updated to only use required_headers
    headers = req_converter.generate_headers(required_headers)
    
    # Process parameters with filtering
    params = req_converter.generate_params(default_params, ignored_params)
    
    print("\nFiltered Headers:")
    for k, v in headers.items():
        print(f"  {k}: {v}")
    
    print("\nFiltered Parameters:")
    for k, v in params.items():
        print(f"  {k}: {v}")
    
    # Export to a file
    if os.path.exists(output_dir):
        filepath = req_converter.export_function(
            export_folder=output_dir,
            prefix="custom_",
            include_test_code=True,
            is_replace=False
        )
        print(f"\nExported to {filepath}")
else:
    print("No collection loaded or collection has no requests.")

## JSON to Python Syntax Conversion

When converting Postman requests with JSON bodies, we need to ensure that JSON boolean values (`true`/`false`/`null`) are properly converted to Python syntax (`True`/`False`/`None`).

In [ ]:
# Example of JSON to Python syntax conversion
from utils import normalize_json_to_python

# Example JSON body with boolean and null values
json_body = '''[{
  "is_active": true,
  "is_deleted": false,
  "metadata": null
}]'''

# Convert to Python syntax
python_body = normalize_json_to_python(json_body)

print("Original JSON body:")
print(json_body)
print("\nConverted Python body:")
print(python_body)